# TSP INSTANCES

In [4]:
from pyscipopt import Model

m = Model("demo")
x = m.addVar("x", lb=0, ub=10)
m.setObjective(x, "maximize")
m.addCons(x <= 5)
m.optimize()
print("Optimal value:", m.getObjVal())

feasible solution found by trivial heuristic after 0.0 seconds, objective value 0.000000e+00
presolving:
Optimal value: 5.0
(round 1, fast)       0 del vars, 1 del conss, 0 add conss, 1 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (2 rounds: 2 fast, 1 medium, 1 exhaustive):
 1 deleted vars, 1 deleted constraints, 0 added constraints, 1 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
transformed 1/2 original solutions to the transformed problem space
Presolving Time: 0.00

SCIP Status        : problem is solved [optimal solution found]
Solving Time (sec) : 0.01
Solving Nodes      : 0
Primal Bound       : +5.00000000000000e+00 (2 solutions)
Dual Bound         : +5.00000000000000e+00
Gap                : 0.00 %


In [4]:
from pyscipopt import Model, quicksum
import numpy as np

# Number of cities
n = 5

# Generate a random symmetric distance matrix
np.random.seed(42)
dist = np.random.randint(10, 100, size=(n, n))
for i in range(n):
    dist[i, i] = 0
    for j in range(i+1, n):
        dist[j, i] = dist[i, j]

print("Distance matrix:\n", dist)

# Create SCIP model
model = Model("TSP_MTZ")

# Binary variables: x[i,j] = 1 if path goes from i -> j
x = {}
for i in range(n):
    for j in range(n):
        if i != j:
            x[i, j] = model.addVar(vtype="B", name=f"x({i},{j})")

# MTZ ordering variables: u[i] gives position of city i in tour
u = {}
for i in range(n):
    u[i] = model.addVar(lb=0, ub=n-1, vtype="C", name=f"u({i})")

# Objective: minimize travel distance
model.setObjective(
    quicksum(dist[i, j] * x[i, j] for i in range(n) for j in range(n) if i != j),
    "minimize"
)

# Each city has exactly one outgoing edge
for i in range(n):
    model.addCons(quicksum(x[i, j] for j in range(n) if i != j) == 1)

# Each city has exactly one incoming edge
for j in range(n):
    model.addCons(quicksum(x[i, j] for i in range(n) if i != j) == 1)

# MTZ subtour elimination constraints
for i in range(1, n):
    for j in range(1, n):
        if i != j:
            model.addCons(u[i] - u[j] + n * x[i, j] <= n - 1)

# Solve the model
model.optimize()

# Print optimal solution
if model.getStatus() == "optimal":
    print(f"\nOptimal tour length = {model.getObjVal():.2f}")

    # Extract tour edges
    tour = []
    current = 0
    visited = {0}
    for _ in range(n):
        for j in range(n):
            if current != j and (current, j) in x and model.getVal(x[current, j]) > 0.5:
                tour.append((current, j))
                current = j
                break
    print("Tour edges:", tour)


Distance matrix:
 [[ 0 24 81 70 30]
 [24  0 84 84 97]
 [81 84  0 62 11]
 [70 84 62  0 73]
 [30 97 11 73  0]]
presolving:

Optimal tour length = 211.00(round 1, fast)       1 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 10 clqs
   (0.0s) running MILP presolver
   (0.2s) MILP presolver found nothing
(round 2, exhaustive) 1 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 10 upgd conss, 0 impls, 10 clqs
   (0.2s) probing cycle finished: starting next cycle
   (0.2s) symmetry computation started: requiring (bin +, int -, cont +), (fixed: bin -, int +, cont -)
   (0.2s) no symmetry present
presolving (3 rounds: 3 fast, 2 medium, 2 exhaustive):
 1 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 24 implications, 10 cliques
presolved problem has 24 variables (20 bin, 0 int, 0 impl, 4 cont) and 22 constraints
     10 constraints

In [9]:
from pyscipopt import Model, quicksum
import numpy as np
import pandas as pd
import random

def solve_tsp_mtz(n, dist, instance_id):
    """
    Solve TSP with MTZ constraints using SCIP.
    Returns objective value and edges in the optimal tour.
    """
    model = Model(f"TSP_MTZ_{instance_id}")
    
    # Decision variables
    x = {}
    for i in range(n):
        for j in range(n):
            if i != j:
                x[i, j] = model.addVar(vtype="B", name=f"x({i},{j})")

    u = {}
    for i in range(n):
        u[i] = model.addVar(lb=0, ub=n-1, vtype="C", name=f"u({i})")

    # Objective
    model.setObjective(
        quicksum(dist[i, j] * x[i, j] for i in range(n) for j in range(n) if i != j),
        "minimize"
    )

    # Degree constraints
    for i in range(n):
        model.addCons(quicksum(x[i, j] for j in range(n) if i != j) == 1)
    for j in range(n):
        model.addCons(quicksum(x[i, j] for i in range(n) if i != j) == 1)

    # MTZ constraints
    for i in range(1, n):
        for j in range(1, n):
            if i != j:
                model.addCons(u[i] - u[j] + n * x[i, j] <= n - 1)

    # Solve
    model.optimize()

    if model.getStatus() == "optimal":
        obj = model.getObjVal()
        # Extract tour edges
        edges = []
        current = 0
        visited = {0}
        for _ in range(n):
            for j in range(n):
                if current != j and (current, j) in x and model.getVal(x[current, j]) > 0.5:
                    edges.append((current, j))
                    current = j
                    break
        return obj
    else:
        return None, None


def generate_instances(num_instances=100, min_cities=5, max_cities=10, seed=42):
    """
    Generate and solve multiple random TSP instances.
    Returns a pandas DataFrame with summary results.
    """
    random.seed(seed)
    np.random.seed(seed)

    results = []
    for inst in range(num_instances):
        n = random.randint(min_cities, max_cities)
        dist = np.random.randint(10, 100, size=(n, n))
        for i in range(n):
            dist[i, i] = 0
            for j in range(i+1, n):
                dist[j, i] = dist[i, j]

        obj = solve_tsp_mtz(n, dist, inst)
        if obj is not None:
            results.append({
                "instance": inst,
                "cities": n,
                "objective": obj
            })
            print(f"Instance {inst}: {n} cities, optimal cost = {obj:.2f}")
        else:
            print(f"Instance {inst}: solver failed")

    return pd.DataFrame(results)


# Example: generate 10 small TSP instances
df = generate_instances(num_instances=10, min_cities=5, max_cities=7)

print(df.head())


presolving:
Instance 0: 7 cities, optimal cost = 261.00
Instance 1: 5 cities, optimal cost = 196.00
Instance 2: 5 cities, optimal cost = 317.00
Instance 3: 7 cities, optimal cost = 158.00
(round 1, fast)       1 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 14 clqs
   (0.0s) running MILP presolver
   (0.0s) MILP presolver found nothing
(round 2, exhaustive) 1 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 14 upgd conss, 0 impls, 14 clqs
   (0.0s) probing cycle finished: starting next cycle
   (0.0s) symmetry computation started: requiring (bin +, int -, cont +), (fixed: bin -, int +, cont -)
   (0.0s) no symmetry present
presolving (3 rounds: 3 fast, 2 medium, 2 exhaustive):
 1 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 60 implications, 14 cliques
presolved problem has 48 variables (42 bin, 0 int, 0 impl, 6 cont) 

 1 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 40 implications, 12 cliques
presolved problem has 35 variables (30 bin, 0 int, 0 impl, 5 cont) and 32 constraints
     12 constraints of type <setppc>
     20 constraints of type <linear>
transformed objective value is always integral (scale: 1)
Presolving Time: 0.00

 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts |sepa|confs|strbr|  dualbound   | primalbound  |  gap   | compl. 
p 0.0s|     1 |     0 |     0 |     - |   locks|   0 |  35 |  34 |  32 |   0 |  0 |   1 |   0 | 0.000000e+00 | 3.620000e+02 |    Inf | unknown
p 0.0s|     1 |     0 |     0 |     - | vbounds|   0 |  35 |  34 |  32 |   0 |  0 |   3 |   0 | 0.000000e+00 | 3.200000e+02 |    Inf | unknown
  0.0s|     1 |     0 |    17 |     - |  1202k |   0 |  35 |  34 |  32 |   0 |  0 |   3 |   0 | 2.093333e+02 | 3.200000e+02 |  52.87%| unknown
r 0.0s|     1 |     0 

# KNAPSACK (0/1) INSTANCES

In [16]:
#Suppose we have a instance of 5 items


import random
from pyscipopt import Model, quicksum

# -----------------------------
# 1. Generate Random Instance
# -----------------------------
def generate_knapsack_instance(num_items=5, max_weight=20, max_value=50, seed=40):
    random.seed(seed)
    weights = [random.randint(1, max_weight) for _ in range(num_items)]
    values = [random.randint(1, max_value) for _ in range(num_items)]
    capacity = int(sum(weights) * 0.6)  # set capacity to 60% of total weight
    return weights, values, capacity

# -----------------------------
# 2. Build and Solve in SCIP
# -----------------------------
def solve_knapsack(weights, values, capacity):
    n = len(weights)
    model = Model("0-1_Knapsack")

    # Decision variables: x[i] = 1 if item i is chosen
    x = {i: model.addVar(vtype="B", name=f"x_{i}") for i in range(n)}

    # Objective: maximize value
    model.setObjective(quicksum(values[i] * x[i] for i in range(n)), "maximize")

    # Weight constraint
    model.addCons(quicksum(weights[i] * x[i] for i in range(n)) <= capacity)

    # Solve
    model.optimize()

    # Extract solution
    solution = {i: model.getVal(x[i]) for i in range(n)}
    total_value = sum(values[i] * solution[i] for i in range(n))
    total_weight = sum(weights[i] * solution[i] for i in range(n))
    
    return solution, total_value, total_weight

# -----------------------------
# Run everything
# -----------------------------
weights, values, capacity = generate_knapsack_instance()
solution, total_value, total_weight = solve_knapsack(weights, values, capacity)

print("Item Weights:", weights)
print("Item Values :", values)
print("Knapsack Capacity:", capacity)
print("\nChosen Items (1 = taken, 0 = not taken):")
for i in range(len(weights)):
    print(f"Item {i}: x={int(solution[i])}, weight={weights[i]}, value={values[i]}")
print(f"\nTotal Weight = {total_weight}")
print(f"Total Value  = {total_value}")


Item Weights:feasible solution found by trivial heuristic after 0.0 seconds, objective value 0.000000e+00
presolving:
 [15, 19, 17, 2, 8]
Item Values : [19, 43, 41, 43, 14]
Knapsack Capacity: 36

Chosen Items (1 = taken, 0 = not taken):
Item 0: x=1, weight=15, value=19
Item 1: x=1, weight=19, value=43
Item 2: x=0, weight=17, value=41
Item 3: x=1, weight=2, value=43
Item 4: x=0, weight=8, value=14

Total Weight = 36.0
Total Value  = 105.0
   (0.0s) running MILP presolver
   (0.0s) MILP presolver found nothing
(round 1, exhaustive) 0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 1 upgd conss, 0 impls, 0 clqs
(round 2, fast)       0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 1 chg sides, 1 chg coeffs, 1 upgd conss, 0 impls, 0 clqs
(round 3, medium)     0 del vars, 1 del conss, 0 add conss, 5 chg bounds, 1 chg sides, 1 chg coeffs, 1 upgd conss, 0 impls, 0 clqs
presolving (4 rounds: 4 fast, 3 medium, 2 exhaustive):
 5 deleted vars, 1 deleted constraints

In [17]:
#Let's solve the LP relaxation

import random
from pyscipopt import Model, quicksum

# -----------------------------
# 1) Generate Random Instance
# -----------------------------
def generate_knapsack_instance(num_items=5, max_weight=20, max_value=50, seed=40):
    random.seed(seed)
    weights = [random.randint(1, max_weight) for _ in range(num_items)]
    values = [random.randint(1, max_value) for _ in range(num_items)]
    capacity = int(sum(weights) * 0.6)  # 60% of total weight
    return weights, values, capacity

# -----------------------------------------
# 2) LP Relaxation (x_i ∈ [0,1], continuous)
# -----------------------------------------
def solve_knapsack_lp_relaxation(weights, values, capacity):
    n = len(weights)
    model = Model("Knapsack_LP_Relax")

    # LP variables: 0 <= x[i] <= 1 (continuous)
    x = {i: model.addVar(lb=0.0, ub=1.0, vtype="C", name=f"x_{i}") for i in range(n)}

    # Maximize total value
    model.setObjective(quicksum(values[i] * x[i] for i in range(n)), "maximize")

    # Capacity constraint
    model.addCons(quicksum(weights[i] * x[i] for i in range(n)) <= capacity)

    # Solve
    model.optimize()

    # Extract (possibly fractional) solution
    sol = {i: model.getVal(x[i]) for i in range(n)}
    total_value = sum(values[i] * sol[i] for i in range(n))
    total_weight = sum(weights[i] * sol[i] for i in range(n))
    return sol, total_value, total_weight

# -----------------------------
# Run everything
# -----------------------------

weights, values, capacity = generate_knapsack_instance()
solution, total_value, total_weight = solve_knapsack_lp_relaxation(weights, values, capacity)

print("Item Weights:", weights)
print("Item Values :", values)
print("Knapsack Capacity:", capacity)
print("\nChosen Fractions (LP relaxation):")
for i in range(len(weights)):
    print(f"Item {i}: x={solution[i]:.4f}, weight={weights[i]}, value={values[i]}")
print(f"\nTotal Weight = {total_weight:.4f}")
print(f"Total Value  = {total_value:.4f}")


Item Weights: [15, 19, 17, 2, 8]
Item Values : [19, 43, 41, 43, 14]
Knapsack Capacity: 36

Chosen Fractions (LP relaxation):
Item 0: x=0.0000, weight=15, value=19
Item 1: x=0.8947, weight=19, value=43
Item 2: x=1.0000, weight=17, value=41
Item 3: x=1.0000, weight=2, value=43
Item 4: x=0.0000, weight=8, value=14

Total Weight = 36.0000
Total Value  = 122.4737
feasible solution found by trivial heuristic after 0.0 seconds, objective value 0.000000e+00
presolving:
(round 1, fast)       4 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       4 del vars, 1 del conss, 0 add conss, 1 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (3 rounds: 3 fast, 1 medium, 1 exhaustive):
 5 deleted vars, 1 deleted constraints, 0 added constraints, 1 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
transformed 1/2 original solutions to the transformed 

In [ ]:
#FIRST LETS TRY TO FIND OPTIMAL SOLUTION THROUGH ENUMERATION

remaining_nodes = []
remaining_items = [0, 1, 2, 3, 4] #5 items

while len(remaining_items) > 0 and len(remaining_nodes) > 0: #while there still exist items not visited
    
    #1) randomly pick an item in remaining items and remove it from remaining items
    item = random.choice(remaining_items)
    remaining_items.remove(item)

    #2) then branch on remaining nodes with that item -- solve the LP relaxation for both items
    

#3) 3 cases:
    # if LP relaxation is integer feasible: add optimal values and objective value to dictionary of optimal solutions
    # if LP relaxation is fractional -- add it to a list of remaining nodes (dictionary of item to constraint)
    

In [40]:
#LP relaxation

weights, values, capacity = generate_knapsack_instance()
solution, total_value, total_weight = solve_knapsack_lp_relaxation(weights, values, capacity)

print("Item Weights:", weights)
print("Item Values :", values)
print("Knapsack Capacity:", capacity)
print("\nChosen Fractions (LP relaxation):")
for i in range(len(weights)):
    print(f"Item {i}: x={solution[i]:.4f}, weight={weights[i]}, value={values[i]}")
print(f"\nTotal Weight = {total_weight:.4f}")
print(f"Total Value  = {total_value:.4f}")

feasible solution found by trivial heuristic after 0.0 seconds, objective value 0.000000e+00
presolving:
Item Weights: [15, 19, 17, 2, 8]
Item Values : [19, 43, 41, 43, 14]
Knapsack Capacity: 36

Chosen Fractions (LP relaxation):
Item 0: x=0.0000, weight=15, value=19
Item 1: x=0.8947, weight=19, value=43
Item 2: x=1.0000, weight=17, value=41
Item 3: x=1.0000, weight=2, value=43
Item 4: x=0.0000, weight=8, value=14

Total Weight = 36.0000
Total Value  = 122.4737
(round 1, fast)       4 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       4 del vars, 1 del conss, 0 add conss, 1 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (3 rounds: 3 fast, 1 medium, 1 exhaustive):
 5 deleted vars, 1 deleted constraints, 0 added constraints, 1 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
transformed 1/2 original solutions to the transformed 

In [43]:
#choose first item to branch on (fractional) -- item 1 is the only fractional one

n = len(weights)
model = Model("Knapsack_LP_Relax")

# LP variables: 0 <= x[i] <= 1 (continuous)
x = {i: model.addVar(lb=0.0, ub=1.0, vtype="C", name=f"x_{i}") for i in range(n)}

# Maximize total value
model.setObjective(quicksum(values[i] * x[i] for i in range(n)), "maximize")

# Capacity constraint
model.addCons(quicksum(weights[i] * x[i] for i in range(n)) <= capacity)

# ✅ NEW: Force item 1 to be 0
model.addCons(x[1] == 1)

# Solve
model.optimize()

# Extract (possibly fractional) solution
sol = {i: model.getVal(x[i]) for i in range(n)}
total_value = sum(values[i] * sol[i] for i in range(n))
total_weight = sum(weights[i] * sol[i] for i in range(n))

print(sol)
print(total_value)
print(total_weight)

#---------------------#

n = len(weights)
model = Model("Knapsack_LP_Relax")

# LP variables: 0 <= x[i] <= 1 (continuous)
x = {i: model.addVar(lb=0.0, ub=1.0, vtype="C", name=f"x_{i}") for i in range(n)}

# Maximize total value
model.setObjective(quicksum(values[i] * x[i] for i in range(n)), "maximize")

# Capacity constraint
model.addCons(quicksum(weights[i] * x[i] for i in range(n)) <= capacity)

# ✅ NEW: Force item 1 to be 0
model.addCons(x[1] == 0)

# Solve
model.optimize()

# Extract (possibly fractional) solution
sol = {i: model.getVal(x[i]) for i in range(n)}
total_value = sum(values[i] * sol[i] for i in range(n))
total_weight = sum(weights[i] * sol[i] for i in range(n))

print(sol)
print(total_value)
print(total_weight)

presolving:
{0: 0.0, 1: 1.0, 2: 0.8823529411764706, 3: 1.0, 4: 0.0}
122.17647058823529
36.0
{0: 0.6, 1: 0.0, 2: 1.0, 3: 1.0, 4: 1.0}
109.4
36.0
(round 1, fast)       2 del vars, 1 del conss, 0 add conss, 1 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (2 rounds: 2 fast, 1 medium, 1 exhaustive):
 5 deleted vars, 2 deleted constraints, 0 added constraints, 2 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
transformed 1/1 original solutions to the transformed problem space
Presolving Time: 0.00

SCIP Status        : problem is solved [optimal solution found]
Solving Time (sec) : 0.00
Solving Nodes      : 0
Primal Bound       : +1.22176470588235e+02 (1 solutions)
Dual Bound         : +1.22176470588235e+02
Gap                : 0.00 %
feasible solution found by trivial heuristic after 0.0 seconds, objective value 0.000000e+00
presolving:
(round 1, fast)       2 del vars, 1 del conss, 0 add conss, 1 chg boun

In [44]:
#2.1 item -- item 0

n = len(weights)
model = Model("Knapsack_LP_Relax")

# LP variables: 0 <= x[i] <= 1 (continuous)
x = {i: model.addVar(lb=0.0, ub=1.0, vtype="C", name=f"x_{i}") for i in range(n)}

# Maximize total value
model.setObjective(quicksum(values[i] * x[i] for i in range(n)), "maximize")

# Capacity constraint
model.addCons(quicksum(weights[i] * x[i] for i in range(n)) <= capacity)

# ✅ NEW: Force item 1 to be 0
model.addCons(x[1] == 0)
model.addCons(x[0] == 1)

# Solve
model.optimize()

# Extract (possibly fractional) solution
sol = {i: model.getVal(x[i]) for i in range(n)}
total_value = sum(values[i] * sol[i] for i in range(n))
total_weight = sum(weights[i] * sol[i] for i in range(n))

print(sol)
print(total_value)
print(total_weight)

#---------------------#

n = len(weights)
model = Model("Knapsack_LP_Relax")

# LP variables: 0 <= x[i] <= 1 (continuous)
x = {i: model.addVar(lb=0.0, ub=1.0, vtype="C", name=f"x_{i}") for i in range(n)}

# Maximize total value
model.setObjective(quicksum(values[i] * x[i] for i in range(n)), "maximize")

# Capacity constraint
model.addCons(quicksum(weights[i] * x[i] for i in range(n)) <= capacity)

# ✅ NEW: Force item 1 to be 0
model.addCons(x[1] == 0)
model.addCons(x[0] == 0)

# Solve
model.optimize()

# Extract (possibly fractional) solution
sol = {i: model.getVal(x[i]) for i in range(n)}
total_value = sum(values[i] * sol[i] for i in range(n))
total_weight = sum(weights[i] * sol[i] for i in range(n))

print(sol)
print(total_value)
print(total_weight)

presolving:
{0: 1.0, 1: 0.0, 2: 1.0, 3: 1.0, 4: 0.25}
106.5
36.0
{0: 0.0, 1: 0.0, 2: 1.0, 3: 1.0, 4: 1.0}
98.0
27.0
(round 1, fast)       3 del vars, 2 del conss, 0 add conss, 2 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       4 del vars, 2 del conss, 0 add conss, 2 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 3, fast)       4 del vars, 3 del conss, 0 add conss, 3 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (4 rounds: 4 fast, 1 medium, 1 exhaustive):
 5 deleted vars, 3 deleted constraints, 0 added constraints, 3 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
transformed 1/1 original solutions to the transformed problem space
Presolving Time: 0.00

SCIP Status        : problem is solved [optimal solution found]
Solving Time (sec) : 0.01
Solving Nodes      : 0
Primal Bound       : +1.06500000000000e+02 (1 solutions)
Dual Bound  

In [45]:
#2.2 item -- item 2

n = len(weights)
model = Model("Knapsack_LP_Relax")

# LP variables: 0 <= x[i] <= 1 (continuous)
x = {i: model.addVar(lb=0.0, ub=1.0, vtype="C", name=f"x_{i}") for i in range(n)}

# Maximize total value
model.setObjective(quicksum(values[i] * x[i] for i in range(n)), "maximize")

# Capacity constraint
model.addCons(quicksum(weights[i] * x[i] for i in range(n)) <= capacity)

# ✅ NEW: Force item 1 to be 0
model.addCons(x[1] == 1)
model.addCons(x[2] == 1)

# Solve
model.optimize()

# Extract (possibly fractional) solution
sol = {i: model.getVal(x[i]) for i in range(n)}
total_value = sum(values[i] * sol[i] for i in range(n))
total_weight = sum(weights[i] * sol[i] for i in range(n))

print(sol)
print(total_value)
print(total_weight)

#---------------------#

n = len(weights)
model = Model("Knapsack_LP_Relax")

# LP variables: 0 <= x[i] <= 1 (continuous)
x = {i: model.addVar(lb=0.0, ub=1.0, vtype="C", name=f"x_{i}") for i in range(n)}

# Maximize total value
model.setObjective(quicksum(values[i] * x[i] for i in range(n)), "maximize")

# Capacity constraint
model.addCons(quicksum(weights[i] * x[i] for i in range(n)) <= capacity)

# ✅ NEW: Force item 1 to be 0
model.addCons(x[1] == 1)
model.addCons(x[2] == 0)

# Solve
model.optimize()

# Extract (possibly fractional) solution
sol = {i: model.getVal(x[i]) for i in range(n)}
total_value = sum(values[i] * sol[i] for i in range(n))
total_weight = sum(weights[i] * sol[i] for i in range(n))

print(sol)
print(total_value)
print(total_weight)

presolving:
{0: 0.0, 1: 1.0, 2: 1.0, 3: 0.0, 4: 0.0}
84.0
36.0
{0: 0.4666666666666667, 1: 1.0, 2: 0.0, 3: 1.0, 4: 1.0}
108.86666666666667
36.0
(round 1, fast)       2 del vars, 2 del conss, 0 add conss, 2 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (2 rounds: 2 fast, 1 medium, 1 exhaustive):
 5 deleted vars, 3 deleted constraints, 0 added constraints, 5 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
transformed 1/1 original solutions to the transformed problem space
Presolving Time: 0.01

SCIP Status        : problem is solved [optimal solution found]
Solving Time (sec) : 0.02
Solving Nodes      : 0
Primal Bound       : +8.40000000000000e+01 (1 solutions)
Dual Bound         : +8.40000000000000e+01
Gap                : 0.00 %
presolving:
(round 1, fast)       2 del vars, 2 del conss, 0 add conss, 2 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       4 del vars,

In [47]:
#3.1 item -- item 4

n = len(weights)
model = Model("Knapsack_LP_Relax")

# LP variables: 0 <= x[i] <= 1 (continuous)
x = {i: model.addVar(lb=0.0, ub=1.0, vtype="C", name=f"x_{i}") for i in range(n)}

# Maximize total value
model.setObjective(quicksum(values[i] * x[i] for i in range(n)), "maximize")

# Capacity constraint
model.addCons(quicksum(weights[i] * x[i] for i in range(n)) <= capacity)

# ✅ NEW: Force item 1 to be 0
model.addCons(x[1] == 0)
model.addCons(x[0] == 1)
model.addCons(x[4] == 1)

# Solve
model.optimize()

# Extract (possibly fractional) solution
sol = {i: model.getVal(x[i]) for i in range(n)}
total_value = sum(values[i] * sol[i] for i in range(n))
total_weight = sum(weights[i] * sol[i] for i in range(n))

print(sol)
print(total_value)
print(total_weight)

#---------------------#

n = len(weights)
model = Model("Knapsack_LP_Relax")

# LP variables: 0 <= x[i] <= 1 (continuous)
x = {i: model.addVar(lb=0.0, ub=1.0, vtype="C", name=f"x_{i}") for i in range(n)}

# Maximize total value
model.setObjective(quicksum(values[i] * x[i] for i in range(n)), "maximize")

# Capacity constraint
model.addCons(quicksum(weights[i] * x[i] for i in range(n)) <= capacity)

# ✅ NEW: Force item 1 to be 0
model.addCons(x[1] == 0)
model.addCons(x[0] == 1)
model.addCons(x[4] == 0)

# Solve
model.optimize()

# Extract (possibly fractional) solution
sol = {i: model.getVal(x[i]) for i in range(n)}
total_value = sum(values[i] * sol[i] for i in range(n))
total_weight = sum(weights[i] * sol[i] for i in range(n))

print(sol)
print(total_value)
print(total_weight)

{0: 1.0, 1: 0.0, 2: 0.6470588235294117, 3: 1.0, 4: 1.0}presolving:

102.52941176470588
36.0
(round 1, fast)       3 del vars, 3 del conss, 0 add conss, 3 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       4 del vars, 4 del conss, 0 add conss, 4 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (3 rounds: 3 fast, 1 medium, 1 exhaustive):
 5 deleted vars, 4 deleted constraints, 0 added constraints, 4 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
transformed 1/1 original solutions to the transformed problem space
Presolving Time: 0.00

SCIP Status        : problem is solved [optimal solution found]
Solving Time (sec) : 0.00
Solving Nodes      : 0
Primal Bound       : +1.02529411764706e+02 (1 solutions)
Dual Bound         : +1.02529411764706e+02
Gap                : 0.00 %
{0: 1.0, 1: 0.0, 2: 1.0, 3: 1.0, 4: 0.0}
103.0
34.0
presolving:
(round 1, fast)       3 del vars

In [48]:
#3.2 item -- item 0

n = len(weights)
model = Model("Knapsack_LP_Relax")

# LP variables: 0 <= x[i] <= 1 (continuous)
x = {i: model.addVar(lb=0.0, ub=1.0, vtype="C", name=f"x_{i}") for i in range(n)}

# Maximize total value
model.setObjective(quicksum(values[i] * x[i] for i in range(n)), "maximize")

# Capacity constraint
model.addCons(quicksum(weights[i] * x[i] for i in range(n)) <= capacity)

# ✅ NEW: Force item 1 to be 0
model.addCons(x[1] == 1)
model.addCons(x[2] == 0)
model.addCons(x[0] == 1)


# Solve
model.optimize()

# Extract (possibly fractional) solution
sol = {i: model.getVal(x[i]) for i in range(n)}
total_value = sum(values[i] * sol[i] for i in range(n))
total_weight = sum(weights[i] * sol[i] for i in range(n))

print(sol)
print(total_value)
print(total_weight)

#---------------------#

n = len(weights)
model = Model("Knapsack_LP_Relax")

# LP variables: 0 <= x[i] <= 1 (continuous)
x = {i: model.addVar(lb=0.0, ub=1.0, vtype="C", name=f"x_{i}") for i in range(n)}

# Maximize total value
model.setObjective(quicksum(values[i] * x[i] for i in range(n)), "maximize")

# Capacity constraint
model.addCons(quicksum(weights[i] * x[i] for i in range(n)) <= capacity)

# ✅ NEW: Force item 1 to be 0
model.addCons(x[1] == 1)
model.addCons(x[2] == 0)
model.addCons(x[0] == 0)

# Solve
model.optimize()

# Extract (possibly fractional) solution
sol = {i: model.getVal(x[i]) for i in range(n)}
total_value = sum(values[i] * sol[i] for i in range(n))
total_weight = sum(weights[i] * sol[i] for i in range(n))

print(sol)
print(total_value)
print(total_weight)

presolving:
{0: 1.0, 1: 1.0, 2: 0.0, 3: 1.0, 4: 0.0}
105.0
36.0
{0: 0.0, 1: 1.0, 2: 0.0, 3: 1.0, 4: 1.0}
100.0
29.0
(round 1, fast)       3 del vars, 3 del conss, 0 add conss, 3 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       4 del vars, 4 del conss, 0 add conss, 4 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (3 rounds: 3 fast, 1 medium, 1 exhaustive):
 5 deleted vars, 4 deleted constraints, 0 added constraints, 4 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
transformed 1/1 original solutions to the transformed problem space
Presolving Time: 0.00

SCIP Status        : problem is solved [optimal solution found]
Solving Time (sec) : 0.01
Solving Nodes      : 0
Primal Bound       : +1.05000000000000e+02 (1 solutions)
Dual Bound         : +1.05000000000000e+02
Gap                : 0.00 %
presolving:
(round 1, fast)       3 del vars, 3 del conss, 0 add conss, 

In [54]:
#3.2 item -- item 0

n = len(weights)
model = Model("Knapsack_LP_Relax")

# LP variables: 0 <= x[i] <= 1 (continuous)
x = {i: model.addVar(lb=0.0, ub=1.0, vtype="C", name=f"x_{i}") for i in range(n)}

# Maximize total value
model.setObjective(quicksum(values[i] * x[i] for i in range(n)), "maximize")

# Capacity constraint
model.addCons(quicksum(weights[i] * x[i] for i in range(n)) <= capacity)

# ✅ NEW: Force item 1 to be 0
model.addCons(x[1] == 0)
model.addCons(x[0] == 1)
model.addCons(x[4] == 1)
model.addCons(x[2] == 0)


# Solve
model.optimize()

# Extract (possibly fractional) solution
sol = {i: model.getVal(x[i]) for i in range(n)}
total_value = sum(values[i] * sol[i] for i in range(n))
total_weight = sum(weights[i] * sol[i] for i in range(n))

print(sol)
print(total_value)
print(total_weight)

#---------------------#

n = len(weights)
model = Model("Knapsack_LP_Relax")

# LP variables: 0 <= x[i] <= 1 (continuous)
x = {i: model.addVar(lb=0.0, ub=1.0, vtype="C", name=f"x_{i}") for i in range(n)}

# Maximize total value
model.setObjective(quicksum(values[i] * x[i] for i in range(n)), "maximize")

# Capacity constraint
model.addCons(quicksum(weights[i] * x[i] for i in range(n)) <= capacity)

# ✅ NEW: Force item 1 to be 0
model.addCons(x[1] == 0)
model.addCons(x[0] == 1)
model.addCons(x[4] == 1)
model.addCons(x[2] == 1)

# Solve
model.optimize()

# Extract (possibly fractional) solution
sol = {i: model.getVal(x[i]) for i in range(n)}
total_value = sum(values[i] * sol[i] for i in range(n))
total_weight = sum(weights[i] * sol[i] for i in range(n))

print(sol)
print(total_value)
print(total_weight)

{0: 1.0, 1: 0.0, 2: 0.0, 3: 1.0, 4: 1.0}
76.0
25.0
presolving:
(round 1, fast)       4 del vars, 4 del conss, 0 add conss, 4 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (2 rounds: 2 fast, 1 medium, 1 exhaustive):
 4 deleted vars, 5 deleted constraints, 0 added constraints, 4 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 1 variables (0 bin, 0 int, 0 impl, 1 cont) and 0 constraints
Presolving Time: 0.00

 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts |sepa|confs|strbr|  dualbound   | primalbound  |  gap   | compl. 
t 0.0s|     1 |     0 |     0 |     - | trivial|   0 |   1 |   0 |   0 |   0 |  0 |   0 |   0 | 7.600000e+01 | 7.600000e+01 |   0.00%| unknown

SCIP Status        : problem is solved [optimal solution found]
Solving Time (sec) : 0.00
Solving Nodes      : 1
Primal Bound       : +7.60000000000000e+01 (1 solutions)
Dual Bound         : +7.6

Warning: SCIPgetSolVal cannot only be called in stage SOLVING without a valid solution (current stage: 10)